In [24]:
from pathlib import Path
path = Path(r"D:\Programme\UBUNTU\Ubuntu.1604.2017.711.0_v1\rootfs\root\com.android.calendar\events-22-55-08\data.csv")

In [25]:
csvdata = []
import csv
with open(path,"r") as fp:
    reader = csv.DictReader(fp,skipinitialspace=True)
    #data = next(reader)
    for row in reader: # Iterate the remaining rows
        csvdata.append(row)
#csvdata

In [26]:
#creates dictonary that contains the events seperated by calendar 
csvcalendars = {} 
# = {calendar1Name: [Event1,...],calendar2Name: [Event1,...]}

for i in range(len(csvdata)):
    key = csvdata[i]['calendar_displayName']  # key = calendarname
    value = csvdata[i]#value = event
    
    if key not in csvcalendars:
        csvcalendars[csvdata[i]['calendar_displayName']] = [value]
    elif type(csvcalendars[key])==list:
         csvcalendars[key].append(value)
    else:
        csvcalendars[key] = [csvcalendars[key][0],value]
csvcalendars

{'S4-R4': [OrderedDict([('sync_data10', 'NULL'),
               ('cal_sync6', 'NULL'),
               ('rrule', 'FREQ=YEARLY;WKST=MO'),
               ('sync_data6', 'NULL'),
               ('cal_sync8', 'NULL'),
               ('eventTimezone', 'Europe/Berlin'),
               ('hasAttendeeData', '1'),
               ('_sync_id', 'NULL'),
               ('customAppPackage', 'NULL'),
               ('originalInstanceTime', 'NULL'),
               ('sync_data2', 'NULL'),
               ('allowedReminders', '0 1'),
               ('uid2445', 'NULL'),
               ('calendar_timezone', 'NULL'),
               ('dirty', '1'),
               ('originalAllDay', 'NULL'),
               ('cal_sync10', 'NULL'),
               ('exrule', 'NULL'),
               ('calendar_color', '-51390'),
               ('cal_sync3', 'NULL'),
               ('lastDate', 'NULL'),
               ('canOrganizerRespond', '1'),
               ('guestsCanSeeGuests', '1'),
               ('rdate', 'NULL'),
        

In [31]:
import icalendar
from icalendar import Calendar, Event, vText,  vDuration
from datetime import datetime
import random

def display(cal):
    return cal.to_ical().decode('utf-8').replace('\r\n', '\n').strip()

In [55]:
import time, pytz #used for time only berlin for now | improve with localize
def epoch_to_timestamp(epochTime):
    return datetime.fromtimestamp(float(epochTime)/1000.).astimezone(pytz.timezone('Europe/Berlin'))
#epoch_to_timestamp('1559174400000')
#def 

datetime.datetime(2019, 5, 30, 2, 0)

In [56]:
 header2componentMap = {
        "title"        :lambda v: ("summary",v),
        "description"  :lambda v:("description",v),
        "eventLocation":lambda v:("location",v), 
        "dtstart"      :lambda time:("dtstart",epoch_to_timestamp(time)),
        "dtend"        :lambda time:("dtend",epoch_to_timestamp(time)),
        #"duration"     :lambda d:("duration",vDuration.from_ical(d)), #doesnt work because it requires a pt infront of time values but there is only a p
        #allday is set when no end or duration is given doesnt work yet sets day from 2am to 2am
        #"rrule"        :lambda rule:("rrule",rule),
    }
def mapHeader2Component(headerKey,value):
    return header2componentMap[headerKey](value)
print(*mapHeader2Component("title","1"))


summary 1


In [57]:
icscalendars = {}
for calendarName in csvcalendars:
    icscalendars[calendarName] = Calendar()
    icscalendars[calendarName].add('prodid', '-//My calendar//')
    icscalendars[calendarName].add('version', '2.0')


def add_events_to_ics(calendarName):
    for csvEvent in csvcalendars[calendarName]:
        print("EName:",csvEvent['title'])
        event = Event()
        event['uid'] = str(random.random())+"@R4"        
        for headerKey in csvEvent:   
            if headerKey in header2componentMap and csvEvent[headerKey]!='NULL':
                print("   "+headerKey+" exists")
            
                event.add(*mapHeader2Component(headerKey,csvEvent[headerKey]))
        icscalendars[calendarName].add_component(event)
#                 event.add(mapHeader2Component.get(headerKey),str(csvEvent[headerKey]).replace(';','lol'))
#                 icscalendars[calendarName].add_component(event)
        
add_events_to_ics("S4-R4")

EName: Subject
   dtstart exists
   description exists
   title exists
   eventLocation exists
EName: Mein 18.
   dtstart exists
   dtend exists
   title exists
   eventLocation exists
EName: AHAHAHAHAHAHAHASUBJECT
   dtstart exists
   dtend exists
   description exists
   title exists
   eventLocation exists


In [58]:
print(display(icscalendars["S4-R4"]))

BEGIN:VCALENDAR
VERSION:2.0
PRODID:-//My calendar//
BEGIN:VEVENT
SUMMARY:Subject
DTSTART;VALUE=DATE-TIME:20190108T080000
UID:0.48880765462675024@R4
DESCRIPTION:Description
LOCATION:Location
END:VEVENT
BEGIN:VEVENT
SUMMARY:Mein 18.
DTSTART;VALUE=DATE-TIME:20190529T020000
DTEND;VALUE=DATE-TIME:20190530T020000
UID:0.7462929225310612@R4
LOCATION:@Home
END:VEVENT
BEGIN:VEVENT
SUMMARY:AHAHAHAHAHAHAHASUBJECT
DTSTART;VALUE=DATE-TIME:20190109T080000
DTEND;VALUE=DATE-TIME:20190109T090000
UID:0.7598117382120483@R4
DESCRIPTION:KEKEKEKKEKEKKEDESCRIPTION
LOCATION:AhahahhaLOCATUON
END:VEVENT
END:VCALENDAR


In [59]:
import tempfile, os
directory = Path(r"")
f = open(os.path.join(directory, 'MYFILEYEEEEEEEEEE.ics'), 'wb') #(filename,mode) wb indicates that the file is opened in binary mode since its a binary file
f.write(icscalendars["S4-R4"].to_ical())
f.close()